In [ ]:
import numpy as np

In [ ]:
fft_step   = 12.5/1000. # 12.5ms

audio_filenames = [ './librivox/guidetomen_%02d_rowland_64kb.mp3' % (i,) for i in [1,2,3]]
audio_filenames

mel_filenames = [ f.replace('.mp3', '.melspectra.hkl') for f in audio_filenames ]

In [ ]:
mel_filename_test = mel_filenames[1]

with open(mel_filename_test.replace('.hkl', '.16_k2.sym'), 'rt') as f:
    mel_sym = f.read()

In [ ]:
def print_one_sec_per_line(s, t_min=0., t_max=None):
    each_line = int(1/fft_step)
    if t_max is None: t_max = len(s)*fft_step
    for t in np.arange(t_min, t_max, 1.):
        i = int(t/fft_step)
        print( s[i:i+each_line] )
        
print_one_sec_per_line(mel_sym, 10.0, 15.0)